In [1]:
##THE HYBRID WITH SLIME AND IALS RESULTS ARE BETTER THAN ANY OTHER ALGORITHM TESTED BEFORE
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices, create_submission, write_submission
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDItemRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeThreeModelsByScores, MergeTwoModelsByScores

In [2]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()
URM_stacked = combine_matrices(ICM=ICM, URM=URM)

In [4]:
##PURE_SVD
PURE_SVD = PureSVDItemRecommender(URM_stacked)
PURE_SVD.fit(num_factors=23,topK=500)

PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!


In [7]:
##SLIMELASTICNET Recommender
SLIME = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_stacked)
SLIME.fit(topK=420, l1_ratio=0.06061216785568925, alpha=0.00554982938879437, workers = 10)


 20%|███████▋                              | 3624/18059 [02:32<11:38, 20.67it/s]


 38%|██████████████▌                       | 6920/18059 [04:57<05:09, 36.03it/s]


 57%|█████████████████████                | 10280/18059 [07:35<11:11, 11.58it/s]


 75%|███████████████████████████▋         | 13544/18059 [10:23<03:55, 19.19it/s]


 93%|██████████████████████████████████▍  | 16808/18059 [13:09<00:48, 25.61it/s]


 99%|████████████████████████████████████▊| 17968/18059 [14:07<00:03, 24.66it/s]

In [9]:
### RP3Beta Recommender
RP32 = RP3betaRecommender(URM_train=URM_stacked)
RP32.fit(topK=int(167.27758470128256), alpha=0.7718223336771766, beta=0.5375685111310673, implicit=True, normalize_similarity=True)


100%|████████████████████████████████████▉| 18056/18059 [14:21<00:00, 24.66it/s]

In [10]:
SLIME_PURESVD = MergeTwoModelsByScores(URM_train =URM_stacked, recommenders= [SLIME,PURE_SVD])
SLIME_PURESVD.fit(alpha=0.790241473254688,beta=0.043735415548593955)

In [12]:
SLIME_PURESVD_RP3 = MergeTwoModelsByScores(URM_train =URM_stacked, recommenders= [SLIME_PURESVD,RP32])
SLIME_PURESVD_RP3.fit(alpha=0.6056872631076499,beta=0.12958301894152272)

In [13]:
submission = create_submission(SLIME_PURESVD_RP3)
write_submission(submission,"SLIME_PURESVD_RP3")

100%|████████████████████████████████████▉| 18056/18059 [15:07<00:00, 19.90it/s]


In [18]:
##SLIMELASTICNET Recommender
SLIME2 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_stacked)
SLIME2.fit(topK=int(526.357696362927), l1_ratio=2.7319880077167894e-05, alpha=0.08228410520589183, workers = 10)

100%|████████████████████████████████████▉| 18032/18059 [16:15<00:00, 36.77it/s]

In [17]:
## IALS Recommender
IALS_recommender = IALSRecommender(URM_train=URM_stacked)
IALS_recommender.fit(epochs=10, num_factors=42, alpha=0.7617528864750021, reg=8.926401306541349)

IALSRecommenderLinear: Epoch 1 of 10. Elapsed time 8.37 sec
IALSRecommenderLinear: Epoch 2 of 10. Elapsed time 18.29 sec
IALSRecommenderLinear: Epoch 3 of 10. Elapsed time 27.89 sec
IALSRecommenderLinear: Epoch 4 of 10. Elapsed time 37.29 sec
IALSRecommenderLinear: Epoch 5 of 10. Elapsed time 45.41 sec
IALSRecommenderLinear: Epoch 6 of 10. Elapsed time 53.11 sec
IALSRecommenderLinear: Epoch 7 of 10. Elapsed time 1.01 min
IALSRecommenderLinear: Epoch 8 of 10. Elapsed time 1.13 min
IALSRecommenderLinear: Epoch 9 of 10. Elapsed time 1.26 min
IALSRecommenderLinear: Epoch 10 of 10. Elapsed time 1.39 min
IALSRecommenderLinear: Terminating at epoch 10. Elapsed time 1.39 min


In [19]:
IALS_RP3 = MergeTwoModelsByScores(URM_train =URM_stacked, recommenders= [IALS_recommender, RP32])
IALS_RP3.fit(alpha=0.5165217586951893,beta=0.8835185422119769)

In [20]:
IALS_RP3_SLIME = MergeTwoModelsByScores(URM_train =URM_stacked, recommenders= [IALS_RP3, SLIME2])
IALS_RP3_SLIME.fit(alpha=0.23588582912644468,beta=1.0)

100%|████████████████████████████████████▉| 18056/18059 [16:32<00:00, 36.77it/s]

In [21]:
submission = create_submission(IALS_RP3_SLIME)
write_submission(submission,"IALS_RP3_SLIME")